In [2]:
import pymysql
import pickle
import json

import math
import numpy as np
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

import os
import time
import hdbscan


In [4]:
path2dir = 'track_communities_data_processing/frac02/'
name_prefix = 'year_clusters_fract02_sample'


In [5]:
# organize samples:

In [8]:
files_list = [f for f in os.listdir(path2dir) if name_prefix in f]
files_list = np.sort(files_list)  # order by sample #

print(files_list)

['year_clusters_fract02_sample0_samples.json'
 'year_clusters_fract02_sample1_samples.json'
 'year_clusters_fract02_sample2_samples.json'
 'year_clusters_fract02_sample3_samples.json'
 'year_clusters_fract02_sample4_samples.json']


In [ ]:
# plot averaged -1 interzones - is there stability?

In [15]:
for fname in files_list:
    print("processing {}".format(fname))
    
    path = path2dir + fname
    with open(path, 'r') as f:
        year_data = json.load(f)
    print("sample fraction: {}".format(year_data['sample_fraction']))
    
    years = []  # get the years included in this sample
    for key in year_data.keys():
        if key.isnumeric():
            years.append(key)
    years = np.sort(years)
    
    for i_year, year in enumerate(years):  # for each year
        print(year_data[year].keys())  #sample_pmids,embedding,labels
        
    

processing year_clusters_fract02_sample0_samples.json
sample fraction: 0.2
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_keys(['sample_pmids', 'embedding', 'labels'])
dict_ke